In [1]:
import PIL.Image
from PIL import Image, ImageTk
from typing import Tuple
from tkinter import *
import cv2

In [2]:
%reload_ext ObjectTracker
%reload_ext PickerApp
from ObjectTracker import ObjectTracker
from PickerApp import PickerApp

stride = 6
margin = 30
scale_factor = 0.5
video = cv2.VideoCapture("videos/full_hd_man.mp4")

root = Tk()
app = PickerApp(root, video, scale_factor)
root.mainloop()

box = app.get_box()
print(box)

ObjectTracker(video, box, stride=stride, margin=margin, scale_factor=scale_factor, debug=True).track()

------------------
483 159
459 696
[318. 966. 426. 600.]
0.014003515243530273
0.01000213623046875
0.008999109268188477
0.009001970291137695
0.008002519607543945
0.01000356674194336
0.011003255844116211
0.011000633239746094
0.00999903678894043
0.009002208709716797
0.00999760627746582
0.012002706527709961
0.009002923965454102
0.009999752044677734
0.01000213623046875
0.011001825332641602
0.009002208709716797
0.009999990463256836
0.010002613067626953
0.009002208709716797
0.010002374649047852
0.011000633239746094
0.011002302169799805
0.00999903678894043
0.014002561569213867
0.009002685546875
0.012002229690551758
0.010002374649047852
0.009002208709716797
0.009001731872558594
0.010002613067626953
0.009002685546875
0.010001897811889648
0.009001493453979492
0.009002685546875
0.008000850677490234
0.009000301361083984
0.009002208709716797
0.010001420974731445
0.009001970291137695
0.0070018768310546875
0.008002042770385742
0.008002519607543945
0.01600337028503418
0.012003183364868164
0.01000237464